In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np

from functools import partial

import tensorflow.contrib.cudnn_rnn as cudnn_rnn
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers
from tensorflow.python.util import nest

In [2]:
#train test btach generator

#Input and output placeholders


#Model

#loss

#Runner


In [4]:
def make_encoder(inputs, hidden_dim, num_stacked_layers, is_train=True, seed=123, output_transpose=True):
    """Build encoder for our RNN model"""
    
    #Expected input shape is batch_size, time sequence len, features
    #params should have - num_layers, rnn_depth, dropout, 
    #Initalize weights

    for i in range(num_stacked_layers):
        with tf.variable_scope('RNN_{}'.format(i)):
            cells.append(tf.contrib.rnn.GRUCell(hidden_dim))
    cell = tf.contrib.rnn.MultiRNNCell(cells)
        
    enc_cell = copy.deepcopy(cell)
    rnn_out, enc_state = tf.contrib.rnn.static_rnn(enc_cell, inputs, dtype=tf.float32)
    c_state=None
    if transpose_output:
        rnn_out = tf.transpose(rnn_out, [1, 0, 2])
        
    return rnn_out, enc_state, c_state

In [5]:
def compressed_readout(rnn_out, attention_depth, dropout, seed):
    """
    FC compression layer, reduces RNN output depth to hparams.attention_depth
    :param rnn_out:
    :param hparams:
    :param dropout:
    :param seed:
    :return:
    """
    if dropout < 1.0:
        rnn_out = tf.nn.dropout(rnn_out, dropout, seed=seed)
    return tf.layers.dense(rnn_out, attention_depth,
                           use_bias=True,
                           activation=tf.nn.selu,
                           kernel_initializer=layers.variance_scaling_initializer(factor=1.0, seed=seed),
                           name='compress_readout'
                           )


In [6]:
def make_fingerprint(x, is_train, fc_dropout, seed):
    """
    Calculates 'fingerprint' of timeseries, to feed into attention layer
    :param x:
    :param is_train:
    :param fc_dropout:
    :param seed:
    :return:
    """
    with tf.variable_scope("fingerpint"):
        # x = tf.expand_dims(x, -1)
        with tf.variable_scope('convnet', initializer=layers.variance_scaling_initializer(seed=seed)):
            c11 = tf.layers.conv1d(x, filters=16, kernel_size=7, activation=tf.nn.relu, padding='same')
            c12 = tf.layers.conv1d(c11, filters=16, kernel_size=3, activation=tf.nn.relu, padding='same')
            pool1 = tf.layers.max_pooling1d(c12, 2, 2, padding='same')
            c21 = tf.layers.conv1d(pool1, filters=32, kernel_size=3, activation=tf.nn.relu, padding='same')
            c22 = tf.layers.conv1d(c21, filters=32, kernel_size=3, activation=tf.nn.relu, padding='same')
            pool2 = tf.layers.max_pooling1d(c22, 2, 2, padding='same')
            c31 = tf.layers.conv1d(pool2, filters=64, kernel_size=3, activation=tf.nn.relu, padding='same')
            c32 = tf.layers.conv1d(c31, filters=64, kernel_size=3, activation=tf.nn.relu, padding='same')
            pool3 = tf.layers.max_pooling1d(c32, 2, 2, padding='same')
            dims = pool3.shape.dims
            pool3 = tf.reshape(pool3, [-1, dims[1].value * dims[2].value])
            if is_train and fc_dropout < 1.0:
                cnn_out = tf.nn.dropout(pool3, fc_dropout, seed=seed)
            else:
                cnn_out = pool3
        with tf.variable_scope('fc_convnet',
                               initializer=layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN', seed=seed)):
            fc_encoder = tf.layers.dense(cnn_out, 512, activation=tf.nn.selu, name='fc_encoder')
            out_encoder = tf.layers.dense(fc_encoder, 16, activation=tf.nn.selu, name='out_encoder')
    return out_encoder